In [1]:
#LGBoost Algorithm
import pandas as pd

In [2]:
dataset=pd.read_csv("insurance_pre.csv")

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
#ONE HOT ENCODING for Nominal Data
dataset=pd.get_dummies(dataset, drop_first=True)
dataset = dataset.astype(int)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27,0,16884,0,1
1,18,33,1,1725,1,0
2,28,33,3,4449,1,0
3,33,22,0,21984,1,0
4,32,28,0,3866,1,0
...,...,...,...,...,...,...
1333,50,30,3,10600,1,0
1334,18,31,0,2205,0,0
1335,18,36,0,1629,0,0
1336,21,25,0,2007,0,0


In [5]:
#SPLITTING INDEPENDENT (INPUT) AND DEPENDENT (OUTPUT) VARIABLE
#To view columns in the dataset
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [10]:
indep=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]
indep

,age,bmi,children,sex_male,smoker_yes
0,19,27,0,0,1
1,18,33,1,1,0
2,28,33,3,1,0
3,33,22,0,1,0
4,32,28,0,1,0
...,...,...,...,...,...
1333,50,30,3,1,0
1334,18,31,0,0,0
1335,18,36,0,0,0
1336,21,25,0,0,0


In [11]:
dep=dataset[['charges']]
dep

,charges
0,16884
1,1725
2,4449
3,21984
4,3866
...,...
1333,10600
1334,2205
1335,1629
1336,2007


In [12]:
import lightgbm as lgb

In [16]:
from sklearn.model_selection import GridSearchCV
param_grid = {'boosting_type':['gbdt'],'num_leaves': [31],'max_depth':[-1], 'learning_rate':[0.1], 'n_estimators': [10, 50,100, 500]}

grid = GridSearchCV(lgb.LGBMRegressor(), param_grid, refit=True, verbose=3, n_jobs=-1)

grid.fit(indep, dep)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'boosting_type': ['gbdt'], 'learning_rate': [0.1],
                         'max_depth': [-1], 'n_estimators': [10, 50, 100, 500],
                         'num_leaves': [31]},
             verbose=3)

In [17]:
# print best parameter after tuning
#print(grid.best_params_)
re=grid.cv_results_
print("The R_score value for best parameter {}:".format(grid.best_params_))

The R_score value for best parameter {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'max_depth': -1, 'n_estimators': 50, 'num_leaves': 31}:


In [18]:
table=pd.DataFrame.from_dict(re)

In [19]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_boosting_type,param_learning_rate,param_max_depth,param_n_estimators,param_num_leaves,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.092103,0.096767,0.004306,0.000571,gbdt,0.1,-1,10,31,"{'boosting_type': 'gbdt', 'learning_rate': 0.1...",0.780174,0.725782,0.760792,0.752772,0.757873,0.755479,0.017510,4
1,0.046822,0.003011,0.004993,0.000404,gbdt,0.1,-1,50,31,"{'boosting_type': 'gbdt', 'learning_rate': 0.1...",0.869952,0.792300,0.884618,0.841778,0.859795,0.849689,0.031903,1
2,0.084969,0.007782,0.007557,0.001032,gbdt,0.1,-1,100,31,"{'boosting_type': 'gbdt', 'learning_rate': 0.1...",0.858987,0.781375,0.876537,0.835307,0.851119,0.840665,0.032483,2
3,0.501965,0.020320,0.034421,0.012189,gbdt,0.1,-1,500,31,"{'boosting_type': 'gbdt', 'learning_rate': 0.1...",0.819499,0.750945,0.839668,0.814321,0.815197,0.807926,0.029940,3


In [20]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 33
BMI: 2
Children: 1
Sex Male 0 or 1: 1
Smoker Yes 0 or 1: 0


In [21]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[6982.0457419]
